# <center>Agent DQN doszkalany ewolucyjnie</center>
<center><img src="https://media.giphy.com/media/dyde6O8yn4oRh7R1Vk/source.gif" width="460" height="280" /></center>

*<center>Wykonali</center>*
*<center>Jakub Gros, Sebastian Kucharzyk</center>*


## Cel i wstępna analiza problemu
<p style='text-align: justify;'>
Jako cel obraliśmy sobie stworzenie maszynowego odpowiednika człowieka grającego w grę 
komputerową(tzw. agenta), który ucząc się na własnych błędach, będzie osiągał w niej coraz to lepsze 
rezultaty. Do tego celu nadaje się m.in. paradygmat procesu uczenia nazywany uczeniem ze wzmocnieniem,
polegający na umieszczeniu uczącego się agenta w nieznanym mu dotąd środowisku i pozwoleniu na 
wykonywanie dowolnych interakcji z otoczeniem. Agent w trakcie eksploracji doświadcza wielu różnych 
sytuacji zwanych stanami. Znajdując się w danym stanie wybiera, którą z obecnie możliwych akcji 
wykona, za co  finalnie otrzymuje pewną nagrodę lub karę. Wszystkie informacje na temat ruchów i 
nagród przechowuje w pamięci, aby nastepnie na ich podstawie móc oceniać, jaka akcja da mu 
największą nagrodę.
</p>


<p style='text-align: justify;'>
Pozornie wydawałoby się, że rozwiązanie to nadaje się niemal idealnie do sytuacji 
wymagających, aby agent sam zebrał dane, na których następnie będzie się uczył, ale niestety posiada ono 
jedną znaczącą wadę - wraz z upływem czasu ilość danych w pamięci agenta rośnie, co z kolei powoduje 
spowolnienie podejmowania decyzji, a finalnie może nawet doprowadzić do zapełnienia pamięci 
fizycznej urządzenia.
</p>


<p style='text-align: justify;'>
Aby uniknąć problemów związanych z nadmiarem danych, zdecydowaliśmny się na wykorzystanie innego paradygmatu, 
który pozwala zgeneralizować zbiór danych zamiast nieskończenie go poszerzać i przeszukiwać. Dokładniej 
rzecz ujmując, wybór padł na zastosowanie głębokiego Q-learningu, czyli połączenia uczenia 
ze wzmocnieniem z uczeniem głębokim. Polega on na zastąpieniu nieograniczonej pamięci agenta siecią neuronową 
wspomaganą relatywnie małym buforem pamiętającym N ostatnich wyborów. Ponadto po każdej akcji podjętej przez agenta
z buforu wybierany jest mały fragment danych, na których następnie sieć jest dotrenowywana.
Wykonuje się to za pomocą Q-wartości, czyli pewnej funkcji  <b>Q(S, A)</b> szacującej jak bardzo opłaca się 
podjąć akcję A będąc w stanie S. Cały powyższy proces prowadzi do stworzenia modelu Deep Q-Network(DQN).
</p>


<p style='text-align: justify;'>
Tutaj zakres projektu mógłby się zakończyć, ale nie chcieliśmy się ograniczać jedynie do głębokiego Q-learningu, 
więc postanowiliśmy znaleźć zastosowanie dla innego zagadnienia z zakresu stucznej inteligencji - algorytmów 
ewolucyjnych. Patrząc na sposób działania modelu DQN można zauważyć trzy obszary, które potencjalnie nadają się 
do ulepszenia ewolucyjnego:
</p>

1. Kształt sieci neuronowej
2. Wagi połączeń międzyneuronowych
3. Dane przechowywane w pamięci agentów

<p style='text-align: justify;'>
Po dokładnym przeanalizowaniu powyższych możliwości zdecydowaliśmy się odrzucić opcje numer 1 oraz 3. 
O wyeliminowaniu manipulacji kształtem sieci neuronowej zadecydował fakt, że wolimy skupić się na
stworzeniu jednego wariantu sieci i jego dogłębnej analizie niż na pobieżnym tworzeniu wielu modeli tylko 
po to, żeby różniły się budową. Natomiast jeśli chodzi o opcję trzecią, to jej odrzucenie wynika z 
natury DQN, której agent przechowuje w pamięci jedynie mały  fragment z całego zbioru danych, 
na którym był uczony. W takim wypadku używanie go jako podstawy do dalszego ulepszania najprawdopodobniej nie 
prowadziłoby do uzyskania wymiernych korzyści. W ten sposób stwierdziliśmy, że najciekawszym i (oby) 
najefektywniejszym podejściem będzie zastosowanie algorytmu genetycznego operującego na wagach połączeń 
międzyneuronowych.
</p>


## Ocena jakości modelu
<p style='text-align: justify;'>
Aby rzetelnie ocenić jakość modelu, należy najpierw upewnić się, że wyniki przez niego osiągane są stabilne oraz 
wiarygodne. Mianem stabilnych określamy je wtedy, gdy w trakcie rozgrywania kolejnych gier przyjmują one zbliżone 
do siebie wartości. Aby osiągnąć taki rezultat, wymagane jest trenowanie modelu na odpowiedniej 
(nie za małej) liczbie gier. Metodą prób i błędów określiliśmy, że dla różnych modeli w naszym środowisku treningowym
liczba ta oscyluje przeważnie w granicach 200.
</p>


<p style='text-align: justify;'>
Z kolei jeśli chodzi o wiarygodność, to uzyskiwane przez dany model rezultaty uznajemy za wiarygodne wtedy, gdy 
nie są one wypaczone przez losowe warunki początkowe. Istnieją dwa różne podejścia pomagające w zapewnieniu 
wiarygodności wyników:
</p>

1. Trenowanie każdego modeli w identycznych warunków początkowych
2. Zastosowanie pewnej funkcji oceny, np. wyciągnięcia średniej z kilku różnych prób wytrenowania tego samego modelu.

<p style='text-align: justify;'>
Ze względu na to, że ideą głębokiego Q-learningu jest eksploracja środowiska w sposób losowy, nasz wybór musiał 
paść na opcję numer 2.
</p>


<p style='text-align: justify;'>
Zważając na powyższe rozważania zdecydowaliśmy, że każdy z naszych modeli będzie trenowany dokładnie cztery 
razy po dwieście gier. Z rezultatów osiąganych we wszystkich czterech podejściach wyciągniemy średnią, a 
następnie porównamy ją z innymi modelami.

## Etapy działania
Podsumowując przeprowadzoną powyżej analizę, zdecydowaliśmy się podzielić realizację projektu na następujące 
fazy:
1. Implementacja i gromadzenie rezultatów naiwnego agenta, który podejmuje decyzje w sposób losowy. Posłuży on 
jako wyznacznik do porównywania innych modeli.
2. Implementacja i gromadzenie rezultatów agenta wykorzystującego prostą sieć neuronową.
3. Modyfikacja implementacji agenta oraz parametrów sieci neuronowej w celu poprawy osiąganych rezultatów.
4. Wytrenowanie początkowej populacji agentów i zastosowanie algorytmu genetycznego do stworzenia osobnika z 
jak najlepiej dostosowanymi wagami międzyneuronowymi.
5. Porównanie osiągniętych rezultatów i wyciągnięcie wniosków z doświadczenia.


## Zastosowane technologie
<p style='text-align: justify;'>
Kod projektu napisaliśmy w języku Python w wersji 3.6. Sieć neuronowa oraz wszelkie struktury i metody
z nią  związane pochodzą z wysokopoziomowego API Kerasa bazującego na bibliotece TensorFlow. Jeśli zaś 
chodzi o samo środowisko do trenowania agenta, wykorzystaliśmy zestaw narzędzi AI Gym, a dokładniej
środowisko gry Cart Pole. Ponadto aby maksymalnie wykorzystać możliwości sprzętowe zdecydowaliśmy, że 
części kodu wymagające wykonywania sekwencyjnego będą uruchamiane z użyciem procesora, a fragmenty 
możliwe do zrównoleglenia obsłuży karta graficzna. 
</p>


## Dygresja na temat podjętych decyzji
<p style='text-align: justify;'>
Pierwszą wersję modelu stworzyliśmy z myślą o uniwersalności. Zasadniczo była ona zdolna do współpracy z 
większością środowisk dostępnych poprzez interfejs AI Gym. Dzięki temu zabiegowi byliśmy w stanie przeprowadzić 
serię prób pozwalających na wyłonienie najlepszego kandydata do dalszego ulepszania modelu. Testy zaczęliśmy od 
bardziej złożonych środowisk (BipedalWalker-v2) poprzez te średnio skomplikowane (LunarLander-v2) aż po 
najłatwiejsze (CartPole-v1). Niestety okazało się, że nasz sprzęt nie jest w stanie wytrenowaćsieci neuronowej 
dla żadnego z zaawansowanych środowisk w akceptowalnych ramach czasowych.
</p>

<p style='text-align: justify;'>
Mówiąc konkretniej, trenowanie naszego modelu do stopnia, w którym jego wyniki są powtarzalne i pozwalają na 
wyciąganie sensownych wniosków, zajmuje około 10 minut w najprostszym środowisku (CartPole). Natomiast w drugim 
co do złożoności LunarLander czas ten jest niemal pięciokrotnie większy. Idąc dalej, po oszacowaniu, że do 
zakończenia projektu będziemy musieli trenować model w różnych konfiguracjach blisko 100 razy, liczba 75 godzin 
spędzonych na samym oczekiwaniu na wytrenowanie się sieci neuronowej dla LunarLandera byłaby zbyt duża. 
Na tej podstawie zdecydowaliśmy, że najlepszym rozwiązaniem będzie wykorzystanie środowiska Cart Pole.
</p>


## Trenowanie i implementacja

In [ ]:
import random
import gym
import tensorflow as tf
a = random.randint()


In [ ]:
NUM_OF_EPISODES = 200
FRAMES_PER_EPISODE = 1000
BATCH_SIZE = 32
GAME_ID = 'CartPole-v1'
CPU_ID = '/device:CPU:0'
print(0)


### Header

### Header

#### Header

##### Header

In [ ]:
import numpy